In [1]:
import numpy as np
import random
import pyomo as pyo
import pandas as pd
from pyomo.environ import *
import pyomo.gdp as gdp
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from itertools import tee
from collections import defaultdict
import itertools
import time
import os

In [2]:
init_data = pd.read_csv('project 4 dataset/init_data.csv')
init_data

,Unnamed: 0,StateCode,State,nb,nb_idx,Available capacity
0,0,AK,Alaska,['WA'],[46],200
1,1,AL,Alabama,"['FL', 'GA', 'MS', 'TN']","[8, 9, 24, 41]",1344
2,2,AR,Arkansas,"['LA', 'MO', 'MS', 'OK', 'TN', 'TX']","[17, 23, 24, 35, 41, 42]",500
3,3,AZ,Arizona,"['CA', 'CO', 'NM', 'NV', 'UT']","[4, 5, 31, 32, 43]",1500
4,4,CA,California,"['AZ', 'HI', 'NV', 'OR']","[3, 10, 32, 36]",11036
5,5,CO,Colorado,"['AZ', 'KS', 'NE', 'NM', 'OK', 'UT', 'WY']","[3, 15, 28, 31, 35, 43, 49]",600
6,6,CT,Connecticut,"['MA', 'NY', 'RI']","[18, 33, 38]",1000
7,7,DE,Delaware,"['MD', 'NJ', 'PA']","[19, 30, 37]",400
8,8,FL,Florida,"['AL', 'GA']","[1, 9]",4000
9,9,GA,Georgia,"['AL', 'FL', 'NC', 'SC', 'TN']","[1, 8, 26, 39, 41]",1006


In [3]:
# str_list = []
# for i,row in init_data.iterrows():
#     if i < 51:
#         linked_states_idx = row['nb_idx'][1:-1].split(',')
#         for lsi in linked_states_idx:
#             str_list.append(str(i)+'-'+str(int(lsi)))
# #str_list

In [4]:
dates = ['0325','0401','0408','0415','0422','0429','0506','0513','0520','0527']
files = os.listdir('project 4 dataset/prediction/0325')
states_file_names = [s for s in files]
states_file_names.remove('.DS_Store')
states_file_names.sort()

In [5]:
df_dict = {}
pred_dict = {'date': [], 'InvVen_upper' : [], 'InvVen_mean' : [], 'InvVen_lower' : []}
for d in dates:
    df_state_list = []
    for state in states_file_names:
        temp = pd.read_csv('project 4 dataset/prediction/'+d+'/'+state)
        df_state_list.append(temp.iloc[0:7,:])

    df_dict[d] = df_state_list

df_dict['0325'][0]

,date,InvVen_upper,InvVen_mean,InvVen_lower
0,2020-03-25,21.001562,6.547376,0.000000
1,2020-03-26,26.333824,8.865605,0.133125
2,2020-03-27,32.942004,11.795609,0.811305
3,2020-03-28,41.708235,15.426271,1.995588
4,2020-03-29,50.472647,19.873626,2.764706
5,2020-03-30,62.352941,25.298724,4.123346
6,2020-03-31,75.072059,31.783325,5.761765


In [6]:
state_names = []
for state in states_file_names:
    state_names.append(state[:-4])
state_names

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

# Solving 

In [7]:
ij_indices = []
s_ij_values = []
s_jj_values = []

G_values = []
U_values = []

state_pos = defaultdict(list)
    
for i,row in init_data.iterrows():
    if i < 51:
        linked_states_idx = row['nb_idx'][1:-1].split(',')
        for lsi in linked_states_idx:
            ij_indices.append(str(i)+'-'+str(int(lsi)))
    
    s_jj_values.append(row['Available capacity'] * 0.6)
    G_values.append(row['Available capacity'] * 0.6 * 0.6)
    U_values.append(row['Available capacity'] * 0.6 * 0.2)
for i in range(0,51):
    ij_indices.append('51-'+str(i))
    
s_ij_indices = ij_indices[:-51]

for i in range(len(s_ij_indices)):
    s_ij_values.append(0)

In [8]:
for iter_, i in enumerate(s_ij_indices):
    state_num = int(i.split('-')[0])
    state_pos[state_num].append(iter_)

In [9]:
state_pos

defaultdict(list,
            {0: [0],
             1: [1, 2, 3, 4],
             2: [5, 6, 7, 8, 9, 10],
             3: [11, 12, 13, 14, 15],
             4: [16, 17, 18, 19],
             5: [20, 21, 22, 23, 24, 25, 26],
             6: [27, 28, 29],
             7: [30, 31, 32],
             8: [33, 34],
             9: [35, 36, 37, 38, 39],
             10: [40],
             11: [41, 42, 43, 44, 45, 46],
             12: [47, 48, 49, 50, 51, 52],
             13: [53, 54, 55, 56, 57],
             14: [58, 59, 60, 61],
             15: [62, 63, 64, 65],
             16: [66, 67, 68, 69, 70, 71, 72],
             17: [73, 74, 75],
             18: [76, 77, 78, 79, 80],
             19: [81, 82, 83, 84, 85],
             20: [86],
             21: [87, 88, 89],
             22: [90, 91, 92, 93],
             23: [94, 95, 96, 97, 98, 99, 100, 101],
             24: [102, 103, 104, 105],
             25: [106, 107, 108, 109],
             26: [110, 111, 112, 113],
             27: [1

In [10]:
def solveModel(s_ij_values, s_jj_values, U_values, G_values):
    m1 = ConcreteModel()
    m1.dual = Suffix(direction=Suffix.IMPORT)
    
    lambda_value = 0.5
    
    #primal decision variables
    m1.s_ij = Var(s_ij_indices, domain = NonNegativeReals)
    m1.s_jj = Var(range(0,52), domain = NonNegativeReals)
    m1.x_ij = Var(ij_indices, domain = NonNegativeReals)
    G = G_values
    U = U_values
    a_jj = s_jj_values
    a_ij = s_ij_values
        

    #Objective
    m1.OBJ = Objective(expr = lambda_value * sum(m1.s_ij[ij] for ij in s_ij_indices), sense=minimize)
    
    
    #Constraints
    # 1
    m1.Constraint1 = ConstraintList()
    for i in range(0,51):
        m1.Constraint1.add(expr = a_jj[i] - (sum(m1.x_ij[ij] for ij in ij_indices[min(state_pos[i]):max(state_pos[i])+1])) + m1.x_ij['51-'+str(i)] == m1.s_jj[i])
    
    
    # 2
    m1.Constraint2 = ConstraintList()
    m1.Constraint2.add(expr = a_jj[51] - (sum(m1.x_ij['51-'+str(j)] for j in range(0,51))) == m1.s_jj[51])
    
    # 3
    m1.Constraint3 = ConstraintList()
    for iter_, ij in enumerate(s_ij_indices):
        m1.Constraint3.add(expr = a_ij[iter_] + m1.x_ij[ij] == m1.s_ij[ij])
    
    
    # 4
    m1.Constraint4 = ConstraintList()
    for i in range(0,52):
        m1.Constraint4.add(G[i] <= m1.s_jj[i])
    
    #5
    m1.Constraint5 = ConstraintList()
    for iter_, ij in enumerate(s_ij_indices):
        u_pos = int(ij.split('-')[0])
        m1.Constraint5.add(expr = -a_ij[iter_] <= m1.x_ij[ij] <= U[u_pos])
    
    
    
    opt = SolverFactory('glpk')
    opt.solve(m1)

    return m1

In [11]:
# trial run for first stage model

for i in range(0, 10):
    print(i)
    m = solveModel(s_ij_values, s_jj_values, U_values, G_values)
    s_ij_values = list(m.s_ij.get_values().values())
    s_jj_values = list(m.s_jj.get_values().values())
    #upperbound_list.append(m.OBJ())
    U_values = list(np.array(s_jj_values) * 0.2)
    G_values = list(np.array(s_jj_values) * 0.6)
    m.s_jj.display()


0
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to e

    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to exp

    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to exp

    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to exp

    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to exp

    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
    function to express ranged inequality expressions. (called from <ipython-
    input-10-5e470ed5bc5e>:47)
s_jj : Size=52, Index=s_jj_index
    Key : Lower : Value  : Upper : Fixed : Stale : Domain
      0 :     0 :  120.0 :  None : False : False : NonNegativeReals
      1 :     0 :  806.4 :  None : False : False : NonNegativeReals
      2 :     0 :  300.0 :  None : False : False : NonNegativeReals
      3 :     0 :  900.0 :  None : False : False : NonNegativeReals
      4 :     0 : 6621.6 :  None : False : False : NonNegativeReals
      5 :     0 :  360.0 :  None : False : False : NonNegativeReals
      6 :     0 :  600.0 :  None : False : False : NonNegativeReals
      7 :     0 :  240.0 :  None : False : False : NonNegativeReals
      8 :     0 : 2400.0 :  None : False : False : NonNegativeReals
      9 :     0 :  603.6 :  None : False : False : NonNegativeReals
     10 :  

ValueError: Constraint 'Constraint4[1]' does not have a proper value. Found 'True'
Expecting a tuple or equation. Examples:
   sum(model.costs) == model.income
   (0, model.price[item], 50)

In [12]:
m.display()

Model unknown

  Variables:
    s_ij : Size=222, Index=s_ij_index
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
         0-46 :     0 :  -0.0 :  None : False : False : NonNegativeReals
         1-24 :     0 :  -0.0 :  None : False : False : NonNegativeReals
         1-41 :     0 :  -0.0 :  None : False : False : NonNegativeReals
          1-8 :     0 :  -0.0 :  None : False : False : NonNegativeReals
          1-9 :     0 :  -0.0 :  None : False : False : NonNegativeReals
         10-4 :     0 :  -0.0 :  None : False : False : NonNegativeReals
        11-13 :     0 :  -0.0 :  None : False : False : NonNegativeReals
        11-22 :     0 :  -0.0 :  None : False : False : NonNegativeReals
        11-23 :     0 :  -0.0 :  None : False : False : NonNegativeReals
        11-28 :     0 :  -0.0 :  None : False : False : NonNegativeReals
        11-40 :     0 :  -0.0 :  None : False : False : NonNegativeReals
        11-47 :     0 :  -0.0 :  None : False : False : NonNegativeR

In [13]:
def benders_lp(constraint_list):
    b = ConcreteModel()
    b.dual = Suffix(direction=Suffix.IMPORT)

    #primal decision variables
    b.eta = Var(domain=NonNegativeReals)
    
    
    #Objective
    b.OBJ = Objective(expr=b.eta, sense=minimize)
    
    #constraint 
    b.constraints = ConstraintList()
    for cons in constraint_list:
        for state_number in states:
            b.constraints.add(expr= cons[0][state_number][cons[3]] - \
                              (cons[1][state_number] + sum(cons[2][i] for i in state_pos[state_number])) <= b.eta)
    
    #Solve
    opt = SolverFactory('glpk')
    opt.solve(b)
    return b.OBJ()

## Bender's Decomposition

In [14]:
upperbound_list = [float('inf')]
lowerbound_list = [-float('inf')]
constraint_list = []
epsilon = 0.001

for week in dates:
    iter_ = 0
    while upperbound_list[iter_] - lowerbound_list[iter_] >= epsilon:
        
        m = solveModel(s_ij_values, s_jj_values, U_values, G_values)
        s_ij_values = list(m.s_ij.get_values().values())
        s_jj_values = list(m.s_jj.get_values().values())
        upperbound_list.append(m.OBJ())
        U_values = list(np.array(s_jj_values) * 0.2)
        G_values = list(np.array(s_jj_values) * 0.6)
        
        l_i = []
        
        for r in ['InvVen_upper', 'InvVen_mean', 'InvVen_lower']:
            demand = list(df_dict[week])
            constraint_list.append((demand, s_jj_values, s_ij_values, r))
            l_i_temp = benders_lp(constraint_list)
            l_i.append(l_i)
        
        lowerbound_list.append(np.average(l_i, weights = [0.33, 0.33, 0.33]))
        iter_ += 1

TypeError: Relational expression used in an unexpected Boolean context.

        The inequality expression:
            72.0  <=  s_jj[0]
        contains non-constant terms (variables) that were evaluated in an
        unexpected Boolean context at
          File '<ipython-input-11-f6c41de06688>', line 5:
    m = solveModel(s_ij_values, s_jj_values, U_values, G_values)

        Evaluating Pyomo variables in a Boolean context, e.g.
            if expression <= 5:
        is generally invalid.  If you want to obtain the Boolean value of the
        expression based on the current variable values, explicitly evaluate the
        expression using the value() function:
            if value(expression) <= 5:
        or
            if value(expression <= 5):
        